In [1]:
import galsim
import numpy as np
import proplot as pplt

In [ ]:
fig, axs = pplt.subplots(figsize=(7, 7))

def _s(x, c, m):
    return 1.0 / (1.0 + np.exp(-(x-m)/c))

def _approx(order, tol):
    plaw = np.exp(4.8 + 9.2e-4 * np.log(tol/1e-8)**2 - 0.33 * np.log(tol/1e-8) - 0.66 * np.log(order))
    sval = _s(np.log(order), np.log(15), 0.1) * (1.0 - _s(np.log(tol), np.log(1e-2), 0.1))
    return np.maximum(sval * 1.5 + (1.0 - sval) * plaw, 0.55)

m = []
orders = np.arange(1, 31, dtype=int)
tols = np.logspace(-8, 0, 10)

vals = np.zeros((tols.shape[0], orders.shape[0]))
for j, tol in enumerate(tols):
    yv = []
    for i in orders:
        lz = galsim.interpolant.Lanczos(i, gsparams=galsim.GSParams(kvalue_accuracy=tol))
        yv.append(lz._i.urange())
    vals[j, :] = np.array(yv)
    m.append(axs.loglog(orders, yv, label="log10(tol) = %d" % np.log10(tol)))
    axs.loglog(
        orders, 
        _approx(orders, tol), 
        ":", 
        color="k",
    )
axs.legend(m, loc="ur", ncols=4, frameon=False)
axs.format(xlabel="Lanczos order", ylabel="urange [$2\pi$/pixels]")
        

In [ ]:
print("_posflux_conserve_dc = {")
for i in orders:
    lz = galsim.interpolant.Lanczos(i, conserve_dc=True, gsparams=galsim.GSParams(kvalue_accuracy=tol))
    print("    %d: %r," % (i, lz.positive_flux))
print("}")

print("_posflux_no_conserve_dc = {")
for i in orders:
    lz = galsim.interpolant.Lanczos(i, conserve_dc=False, gsparams=galsim.GSParams(kvalue_accuracy=tol))
    print("    %d: %r," % (i, lz.positive_flux))
print("}")

print("_negflux_conserve_dc = {")
for i in orders:
    lz = galsim.interpolant.Lanczos(i, conserve_dc=True, gsparams=galsim.GSParams(kvalue_accuracy=tol))
    print("    %d: %r," % (i, lz.negative_flux))
print("}")

print("_negflux_no_conserve_dc = {")
for i in orders:
    lz = galsim.interpolant.Lanczos(i, conserve_dc=False, gsparams=galsim.GSParams(kvalue_accuracy=tol))
    print("    %d: %r," % (i, lz.negative_flux))
print("}")

In [ ]:
with np.printoptions(floatmode="unique"):

    print("_unit_integrals_no_conserve_dc = {")
    for i in orders:
        lz = galsim.interpolant.Lanczos(i, conserve_dc=False, gsparams=galsim.GSParams(kvalue_accuracy=tol))
        print("    %d: jnp.%s, dtype=float)," % (i, repr(lz.unit_integrals())[:-1]))
    print("}")

    print("_unit_integrals_conserve_dc = {")
    for i in orders:
        lz = galsim.interpolant.Lanczos(i, conserve_dc=True, gsparams=galsim.GSParams(kvalue_accuracy=tol))
        print("    %d: jnp.%s, dtype=float)," % (i, repr(lz.unit_integrals())[:-1]))
    print("}")